# Music-Source-Separation-Training-Inference-Webui For Google Colab

For personal entertainment and non-commercial use only. Prohibited from use in content related to blood, gore, violence, sex, politics.<br>
Author: [Github@KitsuneX07](https://github.com/KitsuneX07) | [Github@SUC-DriverOld](https://github.com/SUC-DriverOld) | Gradio theme: [Gradio Theme](https://huggingface.co/spaces/NoCrypt/miku)<br>
Github: [MSST-WebUI](https://github.com/SUC-DriverOld/MSST-WebUI)

## Introduction

This is a webUI for [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training), which is a music source separation training framework. You can use this webUI to infer the MSST model and UVR VR.Models (The inference code comes from [python-audio-separator](https://github.com/nomadkaraoke/python-audio-separator), and we do some changes on it), and the preset process page allows you to customize the processing flow yourself. When used on the cloud, you don't need to download the model yourself. WebUI will automatically download the model you need. Finally, we also provide some convenient tools such as [SOME: Vocals to MIDI](https://github.com/openvpi/SOME/) in the webUI.

In [1]:
#@title Connect to colab runtime and check GPU
#@markdown # Connect to colab runtime and check GPU
#@markdown

!nvidia-smi

Wed Dec 25 05:23:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title Install python 3.10, ffmpeg
#@markdown # Install python 3.10, ffmpeg
#@markdown

!sudo apt install python3.10
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.7).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
#@title Clone repository and install requirements
#@markdown # Clone repository and install requirements
#@markdown
#@markdown ### You may need to rerun this cell for a few times
#@markdown

# Clone the repository
%cd /content
!git clone https://github.com/SUC-DriverOld/MSST-WebUI.git
%cd /content/MSST-WebUI

# Upgrade pip and setuptools
%pip install --upgrade pip setuptools

# Install torch and other requirements
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
%pip install -r requirements.txt --only-binary=samplerate

# Copy cloud webui files to the root directory
!cp tools/webUI_for_clouds/webUI_for_clouds.py .
!cp tools/webUI_for_clouds/download_models.py .

/content
fatal: destination path 'MSST-WebUI' already exists and is not an empty directory.
/content/MSST-WebUI
Looking in indexes: https://download.pytorch.org/whl/cu121
Processing ./tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
librosa is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
#@title [Optional] Download SOME weights if needed
#@markdown # Download SOME weights if needed
#@markdown
#@markdown ### If you want to use "Vocals to MIDI", download the weights.
#@markdown
#@markdown ### "Vocals to MIDI" use project: [SOME](https://github.com/openvpi/SOME/).
#@markdown

!wget https://huggingface.co/Sucial/MSST-WebUI/resolve/main/SOME_weights/model_steps_64000_simplified.ckpt -O /content/MSST-WebUI/tools/SOME_weights/model_steps_64000_simplified.ckpt

--2024-12-25 05:26:30--  https://huggingface.co/Sucial/MSST-WebUI/resolve/main/SOME_weights/model_steps_64000_simplified.ckpt
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/0e/15/0e159d2a04ccab15d8d8e1f184ffbe6cf2f77fb4a01915bea5dc301a76423516/3a246c7063ec2856097004dcefa2093a8b6ccff7a6cde1cd49e8f3f227ede50e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_steps_64000_simplified.ckpt%3B+filename%3D%22model_steps_64000_simplified.ckpt%22%3B&Expires=1735363590&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNTM2MzU5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzBlLzE1LzBlMTU5ZDJhMDRjY2FiMTVkOGQ4ZTFmMTg0ZmZiZTZjZjJmNzdmYjRhMDE5MTViZWE1ZGMzMDFhNzY0MjM1MTYvM2EyNDZjNzA2M2VjMjg1NjA5NzAwNGRjZWZhMjA5M2E4

In [ ]:
#@title [Optional] Pack and upload your music files to your google drive
#@markdown # Pack and upload your music files to your google drive
#@markdown
#@markdown Ensure that there are no nested folders in your zip file, but rather direct audio files.
#@markdown

from google.colab import drive
drive.mount("/content/drive")
#@markdown Directory where **your zip file** located in, dont miss the slash at the end.
data_dir = "/content/drive/MyDrive/MSST-WebUI/"  #@param {type:"string"}
#@markdown Filename of **your zip file**, for example: "inputs.zip"
zip_filename = "input.zip"  #@param {type:"string"}
ZIP_PATH = data_dir + zip_filename
#@markdown You need to package the audio directly into a zip file, **DO NOT** nest folders in the zip file!
#@markdown The compressed file will be extracted to the `input` directory. If there are files in your `input` directory, they will be deleted. Please backup in advance.
#@markdown

!rm -rf /content/MSST-WebUI/input
!mkdir -p /content/MSST-WebUI/input
!unzip -od /content/MSST-WebUI/input {ZIP_PATH}

In [4]:
#@title [Optional] Save results to google drive
#@markdown # Save results to google drive
#@markdown
#@markdown The results audio files in the output directory will be compressed into a zip file and saved to your google drive.
#@markdown

from google.colab import drive
drive.mount("/content/drive")
#@markdown Path to save the zip file, dont miss the slash at the end.
output_dir = "/content/drive/MyDrive/MSST-WebUI/"  #@param {type:"string"}
#@markdown Filename of the zip file, for example: "Outputs.zip"
zip_filename = "Outputs.zip"  #@param {type:"string"}
ZIP_PATH = output_dir + zip_filename

!mkdir -p {output_dir}
!zip -r "{zip_filename}" /content/MSST-WebUI/results
!cp -vr "{zip_filename}" {output_dir}

Mounted at /content/drive
	zip warning: name not matched: /content/MSST-WebUI/results

zip error: Nothing to do! (try: zip -r Outputs.zip . -i /content/MSST-WebUI/results)
cp: cannot stat 'Outputs.zip': No such file or directory


In [ ]:
#@title [Optional] Delete all result audio files
#@markdown # Click to delete all result audio files
#@markdown
#@markdown ### Warning: Unrecoverable, please operate with caution
#@markdown

!rm -rf /content/MSST-WebUI/results

In [ ]:
#@title Initialize and start WebUI
#@markdown # Initialize and start WebUI
#@markdown
#@markdown After running this cell, you can use your own device to connect to WebUI. The public link will be displayed below.
#@markdown
#@markdown If you want to modify the inference parameters of the preset process, edit the `data/webui_config.json` file.
#@markdown

%cd /content/MSST-WebUI

LANGUAGE = "简体中文" #@param ["Auto", "English", "简体中文", "繁體中文", "日本語", "😊"]
MODEL_DOWNLOAD_LINK = "huggingface.co" #@param ["Auto", "huggingface.co", "hf-mirror.com"]
DEBUG = False #@param {type:"boolean"}

import json
import os
import shutil

# Supported languages
language_dict = {"Auto": "Auto", "简体中文": "zh_CN", "繁體中文": "zh_TW", "English": "en_US", "日本語": "ja_JP", "😊": "emoji"}

# Make dirs if not exist
if not os.path.exists("data"):
    shutil.copytree("data_backup", "data")
if not os.path.exists("configs"):
    shutil.copytree("configs_backup", "configs")
os.makedirs("input", exist_ok=True)
os.makedirs("results", exist_ok=True)

# Modify config file
with open("data/webui_config.json", 'r', encoding="utf-8") as f:
    config = json.load(f)
    config['settings']['language'] = language_dict[LANGUAGE]
    config['settings']['download_link'] = MODEL_DOWNLOAD_LINK
    config['settings']['debug'] = DEBUG
with open("data/webui_config.json", 'w', encoding="utf-8") as f:
    json.dump(config, f, indent=4)

# Start WebUI
!python webUI_for_clouds.py

/content/MSST-WebUI
05:27:49.285 [INFO] [webui/utils.py:68] Console log level set to INFO
05:27:49.360 [INFO] [webUI_for_clouds.py:93] 检测到CUDA, 设备信息: {'cuda0': '0: Tesla T4'}
05:27:49.360 [INFO] [webUI_for_clouds.py:100] WebUI Version: 1.7 preview.4, System: Linux, Machine: x86_64
/usr/local/lib/python3.10/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.38.1, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://094ec1db4e2ca6f0aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
--2024-12-25 05:32:13--  https://huggingface.co/Sucial/MSST-WebUI/resolve/main/All_Models/vocal_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.49, ...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')